In [1]:
import numpy as np
import matplotlib.pyplot as plt
import kcorrect
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u


%matplotlib inline

# this notebook is looking at how the final selection compares to the MGS in each band pass
# 0 sdss.modelMag_u, 
# 1 sdss.modelMag_r, 
# 2 sdss.petroMag_r, 
# 3 spec.z, 
# 4 sdss.petroR50_r, 
# 5 galex.fuv_mag, 
# 6 galex.nuv_mag, 
# 7 sdss.modelMag_u, 
# 8 sdss.modelMag_g,
# 9 sdss.modelMag_r,
# 10 sdss.modelMag_i,
# 11 sdss.modelMag_z
# 12 ukidss.yPetroMag, 
# 13 ukidss.j_1PetroMag,
# 14 ukidss.hPetroMag,
# 15 ukidss.kPetroMag,
# 16 wise.w1mpro ,  
# 17 wise.w2mpro , 
# 18 wise.w3mpro , 
# 19 wise.w4mpro , 
# 20 galex.fuv_magerr,
# 21 galex.nuv_magerr,
# 22 sdss.modelMagErr_u ,
# 23 sdss.modelMagErr_g , 
# 24 sdss.modelMagErr_r,
# 25 sdss.modelMagErr_i, 
# 26 sdss.modelMagErr_z,
# 27 ukidss.yPetroMagErr, 
# 28 ukidss.j_1PetroMagErr, 
# 29 ukidss.hPetroMagErr, 
# 30 ukidss.kPetroMagErr, 
# 31 wise.w1sigmpro, 
# 32 wise.w2sigmpro, 
# 33 wise.w3sigmpro, 
# 34 wise.w4sigmpro

In [2]:
cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Om0=0.3)

In [3]:
galaxies = np.load('/home/calum/Documents/Mphys_data/cat_check/cat_check_properties.npy')
galaxies.shape

(61198, 35)

In [4]:
# load filters before hand - too slow to (and pointless) to load each time

def perform_kcorrect(redshift,magnitudes,mag_ivars):
    # remember to apply AB magnitude offsets
    maggies = pow(10,-0.4*np.array(magnitudes))
    maggies_ivar = pow(0.4*np.log(10)*maggies*np.array(mag_ivars),-2)
    c = kcorrect.fit_nonneg(redshift,maggies,maggies_ivar)
    rm = kcorrect.reconstruct_maggies(c)
    rm0 = kcorrect.reconstruct_maggies(c,redshift=0.)
    #print(magnitudes)
    #print(-2.5*np.log10(rm[1:]),-2.5*np.log10(rm0[1:]))
    #print(-2.5*np.log10(rm[1:]/rm0[1:]))
    return -2.5*np.log10(rm[1:]/rm0[1:]) 

In [5]:
# galex-sdss
gkcorrections = []

kcorrect.load_templates()
kcorrect.load_filters(f='galex_sdss.dat')

for row in galaxies:
    # calculate kcorrections
    ktuple = perform_kcorrect(row[3],row[5:12],row[20:27])
    # store in array
    gkcorrections.append([row[3],ktuple])

In [ ]:
# sdss-ukidss
ukcorrections = []

kcorrect.load_templates()
kcorrect.load_filters(f='galex_sdss.dat')

for row in galaxies:
    # calculate kcorrections
    ktuple = perform_kcorrect(row[3],row[5:12],row[20:27])
    # store in array
    gkcorrections.append([row[3],ktuple])